In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import FunctionTransformer
import pandas as pd
from matplotlib import rcParams


from rdkit.Chem import Descriptors
from rdkit import Chem
from sklearn.compose import ColumnTransformer
from numpy import array

rcParams['figure.dpi'] = 900


In [3]:
train_data = pd.read_csv('SGNN_dataset.csv')
train_data = train_data[train_data['dataset_type'] == 'train']
y_train = train_data['logp']
train_data = pd.DataFrame(train_data['molecule'].map(lambda x: Chem.MolFromSmiles(x)))
train_data.head()

,molecule
2054,<rdkit.Chem.rdchem.Mol object at 0x7f12f644fe60>
2055,<rdkit.Chem.rdchem.Mol object at 0x7f12f644fed0>
2056,<rdkit.Chem.rdchem.Mol object at 0x7f12f644fdf0>
2057,<rdkit.Chem.rdchem.Mol object at 0x7f12f644fd80>
2058,<rdkit.Chem.rdchem.Mol object at 0x7f12f644fd10>


In [4]:
test_data = pd.read_csv('SGNN_dataset.csv')
test_data = test_data[test_data['dataset_type'] == 'test']
y_test = test_data['logp']
test_data = pd.DataFrame(test_data['molecule'].map(lambda x: Chem.MolFromSmiles(x)))
test_data.head()

,molecule
11635,<rdkit.Chem.rdchem.Mol object at 0x7f12f709a110>
11636,<rdkit.Chem.rdchem.Mol object at 0x7f12f709a180>
11637,<rdkit.Chem.rdchem.Mol object at 0x7f12f709a0a0>
11638,<rdkit.Chem.rdchem.Mol object at 0x7f12f709a030>
11639,<rdkit.Chem.rdchem.Mol object at 0x7f12f7099fc0>


In [5]:
ConstDescriptors = {"HeavyAtomCount": Descriptors.HeavyAtomCount,
                    "NHOHCount": Descriptors.NHOHCount,
                    "NOCount": Descriptors.NOCount,
                    "NumHAcceptors": Descriptors.NumHAcceptors,
                    "NumHDonors": Descriptors.NumHDonors,
                    "NumHeteroatoms": Descriptors.NumHeteroatoms,
                    "NumRotatableBonds": Descriptors.NumRotatableBonds,
                    "NumValenceElectrons": Descriptors.NumValenceElectrons,
                    "NumAromaticRings": Descriptors.NumAromaticRings,
                    "NumAliphaticHeterocycles": Descriptors.NumAliphaticHeterocycles,
                    "RingCount": Descriptors.RingCount}

PhisChemDescriptors = {"MW": Descriptors.MolWt,
                       "MR": Descriptors.MolMR,
                       "TPSA": Descriptors.TPSA}

descriptors = {}
descriptors.update(ConstDescriptors)
descriptors.update(PhisChemDescriptors)
descriptors_names_list = [key for key in list(descriptors.keys())]

def mol_dsc_calc(mols):
    return pd.DataFrame({k: f(m) for k, f in descriptors.items()}
             for m in array(mols).ravel())

def descriptors_names(transformer, mol_dsc_calc_obj):
    return [f'{key}' for key in list(descriptors.keys())]

descriptors_transformer = FunctionTransformer(mol_dsc_calc, validate=False, feature_names_out=descriptors_names)

features = ColumnTransformer([('descriptors', descriptors_transformer, [0])])

X_train = features.fit_transform(train_data)
X_test = features.fit_transform(test_data)
print(X_train.shape)
print(X_test.shape)

(9581, 14)
(2053, 14)


In [6]:
import numpy as np
sub_indices = np.ix_([0])
X_train[sub_indices]

array([[ 18.   ,   4.   ,   8.   ,   7.   ,   4.   ,   8.   ,   2.   ,
         96.   ,   2.   ,   1.   ,   3.   , 251.246,  59.353, 120.04 ]])

In [7]:
rfc = RandomForestRegressor(random_state=42)

grid = GridSearchCV(rfc,{'max_depth': [30, 40, 50, 60, 70, 80, 90],
                         'criterion': ['squared_error'],
                         'n_estimators': [250, 300, 350, 400, 450, 500],
                         'max_features': ['sqrt', 'log2']},
                         cv=KFold(n_splits=5), verbose=1, scoring='r2', n_jobs=10)

In [8]:
grid.fit(X_train, y_train)
best_model = grid.best_estimator_
best_model

Fitting 5 folds for each of 84 candidates, totalling 420 fits


RandomForestRegressor(max_depth=40, max_features='sqrt', n_estimators=450,
                      random_state=42)

In [9]:
y_pred = grid.predict(X_test)
q2, rmse = r2_score(y_test, y_pred), mean_squared_error(y_test, y_pred, squared=False)
print(f'Q\N{SUPERSCRIPT TWO}: {q2:.3f}\nRMSE: {rmse:.3f}')

Q²: 0.813
RMSE: 0.811


In [10]:
grid.best_estimator_.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': 40,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 450,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [11]:
print(f'R\N{SUPERSCRIPT TWO}: {grid.best_score_:.3f}')

R²: 0.806


In [12]:
print(f'Q\N{SUPERSCRIPT TWO}: {grid.score(X_test, y_test):.3f}')

Q²: 0.813


In [13]:
from joblib import dump, load
dump(grid, 'RF_RDKit_SGNN_train_Seed_42.joblib', compress=3)

['RF_RDKit_SGNN_train_Seed_42.joblib']